## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

-
-

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

# Алгоритм Гаффмана

В цьому алгоритмі доцільно імплементувати клас node та додаткові функції в Huffman для побудови дерева кодування

In [ ]:
class Node:
    def __init__(self, value, code = None, left = None, right = None) -> None:
        self.value = value
        self.left_child = left
        self.right_child = right
        self.code = code

class Huffman:
    @staticmethod
    def encode(text: str) -> tuple[str, dict[str, str]]:
        length, output_chance_dict, sorted_dict = len(text), {}, []

        for i in text:
            output_chance_dict.setdefault(i, 0)
            output_chance_dict[i] += 1/length

        sorted_dict = {Node(value, key) for key, value in output_chance_dict.items()}

        sorted_dict = sorted(sorted_dict, key = lambda x: x.value)

        # Build tree

        while len(sorted_dict) > 1:
            sorted_dict.append(Node(sorted_dict[0].value + sorted_dict[1].value, left = sorted_dict.pop(0), right = sorted_dict.pop(0)))
            sorted_dict = sorted(sorted_dict, key = lambda x: x.value)

        # Encode characters

        encoding_dictinary = {}

        def recursive_encode(node:Node, existing_code):
            if node.code is not None:
                encoding_dictinary[node.code] = existing_code
                return
            recursive_encode(node.left_child, existing_code + '0')
            recursive_encode(node.right_child, existing_code + '1')

        recursive_encode(sorted_dict[0], '')
        return "".join([encoding_dictinary[i] for i in text]) ,encoding_dictinary

    @staticmethod
    def decode(code: str, coding_dict: dict[str, str]):
        if code == '':
            return list(coding_dict.keys())[0]
        point_now, number, output = '', 0, ''
        while code:
            point_now += code[number]
            for key, value in coding_dict.items():
                if point_now == value:
                    code = code[number + 1:]
                    number = -1
                    output += key
                    point_now = ''
                    break
            number += 1
        return output


# Алгоритм LZW

In [ ]:
class LZW:

    @staticmethod
    def encode(text: str) -> tuple[str, list]:
        if not text:
            return text, []
        if len(text) == 1:
            return '0', [text]
        key_dictinary = sorted(set(text))

        initial_dictinary = key_dictinary.copy()

        length = len(text)
        current_code, output_code = text[0], ''
        current_pos = 1
        while current_pos < length:
            add_code = current_code + text[current_pos]
            if add_code in key_dictinary:
                current_code = add_code
                current_pos += 1
            else:
                output_code += str(key_dictinary.index(add_code[:-1])) + ' '
                key_dictinary.append(add_code)
                current_code = ''
        output_code += str(key_dictinary.index(add_code))
        return output_code, initial_dictinary

    @staticmethod
    def decode(code: str, coding_dict: list) -> str:
        if code == '':
            return ''
        if len(code) == 1:
            return coding_dict[0]
        starter_coding_dict = coding_dict.copy()
        output_text = ''
        current_pos_decode, code = 0, code.split(' ')
        lenght = len(code)
        while current_pos_decode < lenght:
            add_text = int(code[current_pos_decode])
            if add_text < len(coding_dict):
                output_text += coding_dict[add_text]
                current_pos_decode += 1
            else:
                coding_dict = starter_coding_dict.copy()
                current_code = ''
                current_pos_encode, add_code = 0, ''
                while len(coding_dict) <= add_text:     ### Double count
                    if current_pos_encode >= len(output_text):
                        coding_dict.append(add_code + add_code[0])
                    else:
                        add_code = current_code + output_text[current_pos_encode]
                    if add_code in coding_dict:
                        current_code = add_code
                        current_pos_encode += 1
                    else:
                        coding_dict.append(add_code)
                        current_code = ''
        return output_text
